In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os, random
from Bio import SeqIO
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn import preprocessing, model_selection
import altair as alt


## Defining paths for each and every omic

In [2]:
path_root_data = os.path.join ('..', 'Data', 'Extracted', 'Databases')
path_all_fasta = os.path.join (path_root_data, 'fasta_files', 'AllBins')
path_normalised_metabolomics = os.path.join (path_root_data, 'Metabolomics', 'Normalised_Tables')
path_model_save_root = 'Saved_models'


In [3]:
num_of_mags = len([i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')])
SEED = 42
END = 25
num_of_workers = 3
random.seed(SEED)
np.random.seed(SEED)

---
## MAG-related functions
---

In [60]:
# Function that splits each genome into k-mers thus creating even longer sentence (MAG)
# It returns tokenized genome i.e. [kmer, kmer,...]
def split_genome (genome, k = 5):
    new_genome = []
    n = len(genome)
    
    if n-k <=0:
        return genome
    else:
        for i in range(n-k):
            new_genome.append(genome[i:i+k])
        
        return new_genome


def vectorize_one_mag (one_mag, w2v_model):
    
    # We have to generate vectors for each word in one MAG and then create vector representation of that MAG
    # by averaging vectors of its words
    zero_vector = np.zeros(w2v_model.vector_size)
    word_vectors = []
    one_mag_vector = []
    
    for sentence in one_mag:
        for word in sentence:
            if word in w2v_model.wv:
                try:
                    word_vectors.append(w2v_model.wv[word])
                except KeyError:
                    print ('Key Error')
                    continue
    
    if word_vectors:
        word_vectors = np.asarray(word_vectors)
        one_mag_vector = word_vectors.mean (axis=0)
    
    else:
        one_mag_vector = zero_vector
    
    return one_mag_vector


# Function that vectorizes a MAG (document) with a pretrained word2vec model. It returns vector representation of a given MAG
# Vectorization is done by averaging word (k-mer) vectors for the whole document (MAG)
def vectorize_mags (w2v_model, end = 25):
    
    print ('Vectorizing MAGs')
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    list_of_mag_vectors = []
    
    # This was done so that I could work with first 'end' FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # Vectorizing MAGs one by one
                list_of_mag_vectors.append (vectorize_one_mag (one_mag, w2v_model))
    
    print ('Finished vectorizing')
    
    return list_of_mag_vectors
    
    

# If one wants to import MAGs in order to vectorize them, one should use start argument in order to skip first 'start' MAGs
# If one wants to import MAGs to train word2vec model, one should use only end argument, so that first 'end' MAGs are used for training
# Todo: Implement randomisation in picking MAGs for training, and don't use first 'start' MAGs for training
# (create list of indexes from 0 to 1364, use sklearn split train test, then traverse directory and use only MAGs with indexes in train for training w2v)
def import_mags_and_build_model (end = 25, path_all_fasta = path_all_fasta):
    
    print ('Importing MAGs')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # If we do not have a model, we build one
                if i == 0:
                    print ('Building w2v model')
                    # We build our model on the first MAG
                    w2v_model = Word2Vec (sentences = one_mag, size = 100, workers = num_of_workers, seed=SEED)
                
                # Else we just expand its vocabulary
                else:
                    # Now we expand our vocabulary
                    w2v_model.build_vocab (one_mag, update = True)
                    
    print ('Finished building')
    
    return w2v_model


def train_model (w2v_model, epochs, end = 25):
    
    print ('Starting model training')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                
                w2v_model.train (one_mag, total_examples = w2v_model.corpus_count, epochs = epochs)
                    
    print ('Model training finished')
    
    return w2v_model


def visualize_with_pca (data, labels, centers):
    
    pca_model = PCA (n_components = 2, random_state = SEED)
    data_transformed = pca_model.fit_transform (data)
    
    data_transformed = pd.DataFrame (data_transformed)
    data_transformed.columns = ['PC_1', 'PC_2']
    data_transformed['Labels'] = labels
    
    chart_data = alt.Chart(data_transformed).mark_circle().encode(
        alt.X ('PC_1:Q'),
        alt.Y ('PC_2:Q'),
        alt.Color ('Labels:N'),
    )
    
    # This means we are visualising centroids from k_means (there are less centroids that data points)
    if data.shape[0] != centers.shape[0]:
        
        centers_transformed = pca_model.fit_transform (centers)
        centers_transformed = pd.DataFrame (centers_transformed)
        centers_transformed.columns = ['PC_1', 'PC_2']
        
        chart_centers = alt.Chart(centers_transformed).mark_point(shape = 'diamond', color = 'black', size = 50, opacity = 0.7).encode(
            alt.X ('PC_1:Q'),
            alt.Y ('PC_2:Q'),
        )
        
        return chart_data + chart_centers
    
    # For DBSCAN there are no centroids
    else:
        return chart_data



---
## Importing Metabolomic data

---

In [5]:
metabolomics_file_name = os.path.join(path_normalised_metabolomics, os.listdir(path_normalised_metabolomics)[0])
metabolomics_df = pd.read_csv (metabolomics_file_name, delimiter = '\t')
metabolomics_df

,Metabolite,tp,date,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,!Phosphoric_acid_3TMS,D1,2012-04-17,snp,known,0.757113,0.737943,0.098650,3,0.056956,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
1,!Phosphoric_acid_3TMS,D10,2012-01-25,snp,known,0.720935,0.873509,0.266593,3,0.153918,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
2,!Phosphoric_acid_3TMS,D11,2012-03-22,snp,known,0.625047,0.652268,0.057820,3,0.033382,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
3,!Phosphoric_acid_3TMS,D12,2012-01-19,snp,known,0.964956,0.941878,0.053754,3,0.031035,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
4,!Phosphoric_acid_3TMS,D13,2011-05-13,snp,known,0.963984,0.974479,0.071659,3,0.041373,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,Valine_2TMS,D51,2011-08-11,bp,known,0.704821,0.478251,0.428368,3,0.247319,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18866,Valine_2TMS,D6,2011-06-03,bp,known,1.085888,1.197471,0.487484,3,0.281449,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18867,Valine_2TMS,D7,2012-04-04,bp,known,2.958023,2.813145,0.341213,3,0.197000,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18868,Valine_2TMS,D8,2012-02-08,bp,known,2.515980,2.322248,0.618556,3,0.357123,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine


In [6]:
metabolomics_df['date'] = pd.to_datetime(metabolomics_df['date'])
metabolomics_df.insert (0, 'date', metabolomics_df.pop('date'))
metabolomics_df.sort_values ('date', inplace = True, ignore_index = True)
metabolomics_df

,date,Metabolite,tp,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,2011-03-21,Leucine_2TMS,D37,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar,"C16439,C00123,C01570",leucine;L-leucine;D-leucine,leucine;L-leucine;D-leucine
1,2011-03-21,No match: 1181.78_EM_SP_D1_1_223,D37,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar,NaN,NaN,NaN
2,2011-03-21,unknown#emu_WW_2144.25,D37,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar,NaN,NaN,NaN
3,2011-03-21,Unknown#bth_pae_001,D37,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar,NaN,NaN,NaN
4,2011-03-21,No match: 2220.34_EM_BP_D1_1_192,D37,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,Octadecanol_1TMS,D48,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar,D01924,octadecan-1-ol,octadecan-1-ol
18866,2012-05-03,No match: 1447.93_EM_BP_D2_1_57,D48,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar,NaN,NaN,NaN
18867,2012-05-03,Lactulose_8TMS,D48,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar,C07064,lactulose,lactulose
18868,2012-05-03,No match: 2004.78_EM_BNP_D10_1_205,D48,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar,NaN,NaN,NaN


## MAG examination

To do:
1. Make kmers of some length, out of every word in every sentence
2. Encode every sentence with Word2Vec and CBOW (because it's faster than skip-gram)
3. Reduce dimensionality with PCA or with autoencoder
    > Maybe it is good to do the first 3 steps while importing, for each and every sentence. That way, I might use less memory
4. Cluster similar sentences (MAGs) to get rMAGs

> Important: this is not temporal data, since there are no timestamps!


In [7]:

# FOR CLUSTERING I SHOULD CREATE A DATAFRAME WITH MAGs INDEXES AND THEIR VECTOR REPRESENTATIONS

final_model = import_mags_and_build_model (end = END, path_all_fasta = path_all_fasta)


Importing MAGs
Building w2v model
Finished building


In [8]:
final_model.wv.most_similar('ACGGC')

[('CCGGC', 0.5773573517799377),
 ('TCGGC', 0.5635622143745422),
 ('ACGGA', 0.5475510358810425),
 ('GCGGC', 0.5233906507492065),
 ('ACGGT', 0.4797680675983429),
 ('ACGGG', 0.47880542278289795),
 ('ACGAC', 0.29950520396232605),
 ('ACGTT', 0.2790815234184265),
 ('ACGAT', 0.26410460472106934),
 ('AACGA', 0.26403582096099854)]

In [9]:
# Train model. It tooks ~10 minutes for END = 25 amount of MAGs
epochs = 10

final_model = train_model (final_model, epochs = epochs, end = END)


Starting model training
Model training finished


In [10]:
final_model.wv.save_word2vec_format(os.path.join (path_model_save_root, 'model_25.bin'), binary=True) 

Now I should vectorize documents with this model. For further use, I could save this model's weights, and use it to vectorize all mags. That would take a lot, but every MAG will have its vector representation
> This could be done by importing one MAG at a time, then tokenizing it (like before), then getting vector representations of that MAG's sentences (genomes) and then finding the vector representation of the whole MAG (document). If I do that for one MAG at a time, There is no need to worry about memory


In [11]:
list_of_mag_vectors = vectorize_mags (final_model, end = END)

Vectorizing MAGs
Finished vectorizing


In [12]:
list_of_mag_vectors[0:2]

[array([ 1.1176913e+00, -1.2851473e+00,  1.2977009e+00, -3.5607186e-01,
         1.1518677e+00, -1.4784318e+00,  6.9590878e-01, -2.0498614e+00,
         7.7519298e-01,  9.5224845e-01,  8.7902375e-02, -8.6440951e-01,
        -1.4811026e-01,  2.1250219e+00, -6.5102720e-01,  6.4006710e-01,
        -7.7712810e-01, -6.8890983e-01, -1.7285050e+00,  3.5550112e-01,
         7.3951113e-01, -1.9243872e+00,  1.3869661e+00,  1.4437793e-01,
        -1.2514910e+00,  4.9194074e-01,  7.0413029e-01, -2.1186869e-01,
         1.1919310e+00,  4.0589210e-01, -1.1578156e+00, -1.0346288e-01,
        -4.7114155e-01,  2.5510933e+00,  8.9802682e-01, -1.0469451e+00,
        -4.5550957e-01,  5.2043205e-01, -3.8725996e-01,  3.1858125e-01,
         8.2555884e-01, -1.8461175e+00,  5.8819723e-01, -6.8201619e-01,
         2.8457314e-01, -2.1281188e+00,  1.8199693e+00, -6.0000217e-01,
        -8.4681815e-01, -1.4194238e+00, -3.9664999e-01,  1.5638909e-01,
         1.8452522e+00, -3.9363700e-01,  9.9437870e-04,  1.11156

In [13]:
mags_df = pd.DataFrame (list_of_mag_vectors)
mags_df


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.117691,-1.285147,1.297701,-0.356072,1.151868,-1.478432,0.695909,-2.049861,0.775193,0.952248,...,1.548072,-1.212546,1.297358,0.304305,-0.088180,0.741703,-2.192100,-1.760029,1.292709,-0.566314
1,-0.397457,-0.586370,-2.101782,-0.957839,-0.894716,2.252759,0.179345,1.366534,-0.141001,-0.368006,...,-0.324187,0.456942,-1.723317,-1.089818,1.471538,-1.332001,2.797414,0.962129,-0.949379,0.027219
2,1.428277,-0.409841,1.821033,-0.632364,0.519591,-1.075682,0.467286,-2.106847,0.623872,-0.444501,...,0.977349,-2.612781,0.853364,1.045666,-0.377918,1.504324,-3.668192,-3.049183,1.911584,-1.719634
3,-0.807010,-1.132076,-2.040275,-1.018199,-1.241043,2.814131,-0.277895,1.702027,-0.216814,-0.389414,...,-0.576729,0.361689,-1.794586,-0.989517,1.530060,-1.400325,3.109184,1.333641,-1.368727,-0.299508
4,-0.729850,-1.103235,-1.926998,-1.014075,-1.196003,2.693963,-0.228020,1.613018,-0.233582,-0.356466,...,-0.552327,0.412861,-1.779991,-0.922350,1.528533,-1.356465,2.961123,1.244486,-1.314663,-0.330579
5,-0.700547,-0.697610,-0.764258,-0.581024,-1.340819,2.218557,-0.651789,1.153820,-0.019014,-0.771810,...,-0.663449,-0.050688,-1.609203,-0.787775,1.419916,-0.765434,1.982262,0.660961,-1.071012,-0.559409
6,1.882998,-0.430399,2.193910,-0.951240,1.713361,-2.041455,1.248835,-3.099297,0.470355,0.100278,...,1.631961,-2.643044,1.638079,1.352711,-1.153366,2.204770,-5.934611,-4.415373,3.064456,-2.190602
7,1.943645,-1.087292,2.275767,-0.813578,1.121162,-1.855459,1.098353,-3.171567,0.737215,0.383941,...,1.407518,-2.815509,2.073592,1.181931,-0.901339,1.766575,-4.593009,-3.681092,2.426500,-1.965810
8,0.527792,-1.015823,1.037761,-0.317868,-0.009117,-0.300108,0.145670,-1.214258,0.575413,0.319207,...,0.590685,-1.015095,0.404293,-0.036654,0.229922,0.548078,-0.990845,-1.071017,0.159505,-0.640709
9,0.226097,-0.912251,0.708252,-0.288762,0.087465,0.339399,-0.080053,-0.495883,0.601787,-0.025573,...,0.509829,-0.953439,-0.174323,-0.128326,0.652848,0.279634,-0.737565,-0.845867,0.400943,-0.742461


---
## Data preprocessing
---

In [14]:
mag_scaler = preprocessing.StandardScaler()
scaled_mags_df = mag_scaler.fit_transform(mags_df)

scaled_mags_df

array([[ 1.00382113, -1.19856783,  0.90286325, ..., -0.63545758,
         0.85830795,  0.12988954],
       [-0.57417116,  0.35773879, -1.35580206, ...,  0.89849409,
        -0.69996512,  0.95923406],
       [ 1.3272889 ,  0.7509027 ,  1.2505726 , ..., -1.36190356,
         1.28843227, -1.48164563],
       ...,
       [ 1.6245575 ,  0.59847658,  1.21080464, ..., -1.8487907 ,
         1.78209276, -1.71927117],
       [-0.46539595,  2.39491147, -0.12714411, ...,  0.38056967,
        -0.87722101,  0.50305431],
       [-1.37631748, -0.27466974, -1.06073248, ...,  0.81899784,
        -1.07330886, -0.52555183]])

---
## Clustering
---

### 1. K-means

In [15]:
k_range_end = 15 # Usually it is sqrt(# of mags)

k_range = range(1, k_range_end)

k_mean_models = [KMeans (n_clusters = i, random_state = SEED) for i in k_range]
k_scores = [k_mean_model.fit(scaled_mags_df).score(scaled_mags_df) for k_mean_model in k_mean_models]
k_data = pd.DataFrame ({'k_range':k_range, 'k_scores':k_scores})
k_data


,k_range,k_scores
0,1,-2500.000000
1,2,-945.723227
2,3,-595.438301
3,4,-466.881248
4,5,-369.161081
5,6,-294.748963
6,7,-253.504334
7,8,-214.095257
8,9,-186.160850
9,10,-162.447143


In [16]:
k_num_chart = alt.Chart(data = k_data).mark_line().encode(
    alt.X ('k_range:Q'),
    alt.Y ('k_scores:Q')
)

k_num_chart

alt.Chart(...)

In [44]:
# We can see from the chart above that 6 or 7 clusters are optimal for this task (where END = 25 MAGs)
num_of_clusters = 7

k_means_model = KMeans (n_clusters = num_of_clusters, random_state = SEED)
k_means_predicted = k_means_model.fit_predict(scaled_mags_df)
k_means_predicted

array([6, 1, 2, 1, 1, 4, 2, 2, 0, 0, 0, 0, 0, 4, 4, 5, 1, 5, 4, 6, 0, 1,
       2, 3, 4], dtype=int32)

In [61]:
k_means_chart = visualize_with_pca (scaled_mags_df, k_means_predicted, k_means_model.cluster_centers_)
k_means_chart


alt.LayerChart(...)

### 2. DBSCAN

In [52]:
MIN_SAMPLES = 3
EPS = 7

dbscan_model = DBSCAN(eps = EPS, min_samples = MIN_SAMPLES, n_jobs = num_of_workers)
dbscan_predicted = dbscan_model.fit_predict (scaled_mags_df)
dbscan_predicted

array([ 0,  1,  2,  1,  1,  1,  2,  2,  0,  0,  0,  0,  0,  1,  1,  1,  1,
        1,  1,  0,  0,  1,  2, -1,  1])

In [53]:
dbscan_core_samples = np.zeros_like(dbscan_predicted, dtype=bool)
dbscan_core_samples[dbscan_model.core_sample_indices_] = 1
print(dbscan_core_samples)
dbscan_model.core_sample_indices_

[ True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True]


array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 24])

In [54]:
# Visualize clusters
dbscan_chart = visualize_with_pca (scaled_mags_df, dbscan_predicted, dbscan_core_samples)
dbscan_chart


alt.Chart(...)

---
## Evaluation
---